##Importing the LIBRARIES

In [25]:
import openai
import langchain
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
# from langchain import PineconeVectorStore
# from pinecone import Pinecone

Loading the env variables

In [59]:
from dotenv import load_dotenv
load_dotenv()##loads all the environment variables from the .env file
import os

Reading the document from the documents folder

In [17]:
#Read the document
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents

In [18]:
doc = read_doc("documents/")

In [19]:
#Divide the docs into chunks as there is token limitation

def chunk_data(docs, chunk_size = 800, chunk_overlap = 50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = chunk_overlap)
    docs = text_splitter.split_documents(docs)
    return docs

In [ ]:
documents = chunk_data(docs=doc)
documents

In [31]:
# import os

# openai_key = ""
# pinecone_api_key = ""

EMBEDDING the data from the document for uploading on the database.

In [34]:
# Embedding technique of OPENAI

embeddings = OpenAIEmbeddings(
    api_key=openai_api_key,
)

embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001D2F9A56050>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001D2F9A60990>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key='sk-proj-JWNqVzLG40YZcZilB8OlT3BlbkFJf8OBWkPX1StzhqEjkt0l', openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None)

In [35]:
vectors = embeddings.embed_query("How are you?")
len(vectors)#just an example

1536

Upload the embeddings to Pinecone database

In [36]:
from langchain_pinecone import PineconeVectorStore
index_name = "langchainvector"
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

In [37]:
index = PineconeVectorStore.from_documents(doc, index_name=index_name, embedding=embeddings)

Pass the query to the db and get some similar content from the document in the db

In [39]:
##Consine Similarity Retrieve Results
##this is the function to get the data from the vector db

def retrieve_query(query, k=2):
    matching_results = index.similarity_search(query, k=k)
    return matching_results

In [40]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

Initialise the LLM and chain(connecting the steps of query retrieval and the api call to the LLM)

In [43]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.5, max_tokens=100)
chain = load_qa_chain(llm, chain_type="stuff")

c:\Users\Asus\Desktop\Vectordb\env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


Using the chain to connect the steps

In [44]:
## Search answers from vector db
def retrieve_answers(query):
    doc_search = retrieve_query(query)
    print(doc_search)
    response = chain.run(input_documents=doc_search, question=query)
    return response

In [65]:
our_query = "Read the work experience at Nokia and rewrite it professionally"
answer = retrieve_answers(our_query)
print(answer)

[Document(page_content="Dinesh\nBalivada\nBangaluru,\nIndia\n•\n+91\n7993438995\n•\ndineshbalivada8@gmail.com\nLinkedIn\nProfile\n•\nGitHub\nProfile\n—————————————————————————————————————————————————\nE\nDUCATION\nIndian\nInstitute\nof\nTechnology\nMadras\nChennai,\nIndia\nBachelor\nof\nScience,\nData\nScience\nand\nProgramming\n2021-2025\nThe\nNational\nInstitute\nof\nEngineering\nMysuru,\nIndia\nBachelor\nof\nEngineering,\nComputer\nScience\nand\nEngineering\n2020-2024\n—————————————————————————————————————————————————\nW\nORK\nE\nXPERIENCE\nN\nOKIA\nN\nETWORKS\nA\nND\nS\nOLUTIONS\n2024\n|\nBangaluru,\nIndia\n•\nCurrently\nworking\nas\nAugmented\nAutomation\nStudent\nTrainee\nfor\na\nperiod\nof\n6\nmonths.\n•\nHands-on\nexperience\non\nMicrosoft\nPower\nPlatforms\n(including\nPower\nBI)\nand\ntrainings\non\nMicrosoft\nAzure\nCloud\nServices.\n•\nOrchestrated\na\ntransformative\nproject\nleveraging\nPower\nApps\nand\nPower\nAutomate\nsoftware\nto\nincorporate\nLarge\nLanguage\nModels\